## This note book gives the trend of a single word in single mailing list.

In [1]:
%matplotlib inline

In [2]:
import bigbang

In [3]:
from bigbang.archive import Archive
import bigbang.parse as parse
import bigbang.graph as graph
import bigbang.mailman as mailman
import bigbang.process as process
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from pprint import pprint as pp
import pytz
import numpy as np
import math
import nltk
from itertools import repeat
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from nltk.corpus import stopwords
import re

In [4]:
urls = ["http://mail.scipy.org/pipermail/ipython-dev/"]#,
        #"http://mail.scipy.org/pipermail/ipython-user/"],
        #"http://mail.scipy.org/pipermail/scipy-dev/",
        #"http://mail.scipy.org/pipermail/scipy-user/",
        #"http://mail.scipy.org/pipermail/numpy-discussion/"]


archives= [Archive(url,archive_dir="../archives") for url in urls]

In [6]:
checkword = "python" #can change words, should be lower case

In [9]:
df = pd.DataFrame(columns=["MessageId","Date","From","In-Reply-To","Count"])
for row in archives[0].data.iterrows():
    try:
        w = row[1]["Body"].replace("'", "")
        k = re.sub(r'[^\w]', ' ', w)
        k = k.lower()
        t = nltk.tokenize.word_tokenize(k)
        subdict = {}
        count = 0
        for g in t:
            try:
                word = st.stem(g)
            except:
                print g
                pass
            if word == checkword:
                count += 1
        if count == 0:
            continue
        else:
            subdict["MessageId"] = row[0]
            subdict["Date"] = row[1]["Date"]
            subdict["From"] = row[1]["From"]
            subdict["In-Reply-To"] = row[1]["In-Reply-To"]
            subdict["Count"] = count
            df = df.append(subdict,ignore_index=True)
    except:
        print row

('<BAY131-DAV1715D9C1DAB1066F6568A0BA870@phx.gbl>', From                   clarkdbassett@hotmail.... (Clark Bassett)
Subject                                 [IPython-dev] unsuscribe
Date                                         2007-03-02 19:19:33
In-Reply-To                                                 None
References     <mailman.9.1172858404.28084.ipython-dev@scipy....
Body                                                        None
Name: <BAY131-DAV1715D9C1DAB1066F6568A0BA870@phx.gbl>, dtype: object)
('<1F100CE1-726A-41A3-917E-C1D35793E3F8@gmail.com>', From           bussonniermatthias@gmail.... (Matthias BUSSONN...
Subject                                   [IPython-dev] IPEP 10:
Date                                         2013-02-25 17:06:14
In-Reply-To    <CAHNn8BVgLjUwU9aYkjgwnnP=FPXRwJzOKaDKT+dviAZc...
References     <CAHNn8BVgLjUwU9aYkjgwnnP=FPXRwJzOKaDKT+dviAZc...
Body                                                        None
Name: <1F100CE1-726A-41A3-917E-C1D35793E3F8@g

In [ ]:
df[:5]  #dataframe of informations of the particular word.

In [ ]:
start_date = df.iloc[0]["Date"]
end_date = df.iloc[len(df.index)-1]["Date"]

In [ ]:
start_date

In [ ]:
end_date

In [ ]:
total_month = (end_date.year - start_date.year)*12 + (end_date.month-start_date.month+1)

In [ ]:
total_month

In [ ]:
k_month = []
temp_year = start_date.year
temp_month = start_date.month
for i in range(total_month):
    k_month.append((temp_year, temp_month))
    if temp_month == 12:
        temp_year += 1
        temp_month = 0
    temp_month += 1

In [ ]:
time_dict = {}
for row in df.iterrows():
    if (row[1]['Date'].year, row[1]['Date'].month) not in time_dict:
        time_dict[(row[1]['Date'].year, row[1]['Date'].month)] = row[1]["Count"]
    else:
        time_dict[(row[1]['Date'].year, row[1]['Date'].month)] += row[1]["Count"]

In [ ]:
by_month = []
for item in k_month:
    if item in time_dict:
        by_month.append(time_dict[item])
    else:
        by_month.append(0)

In [ ]:
x = range(1,total_month+1)
y = by_month
plt.plot(x, y)
plt.xlabel("month")

In [ ]:
y[62] #highest

In [ ]:
#Check which month is 62
k_month[62]